In [4]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch


# 加载数据
train_data = pd.read_csv('C:/Users/DELL/Downloads/movie_train.csv')
test_data = pd.read_csv('C:/Users/DELL/Downloads/movie_test.csv')

# 初始化tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# 数据预处理函数
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# 创建datasets
train_dataset = Dataset.from_pandas(train_data).map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)

# 加载预训练模型
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# 指定设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # 将模型转移到指定的设备

Map: 100%|██████████| 8000/8000 [00:07<00:00, 1126.06 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:


# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
)

# 初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from nltk.tokenize import word_tokenize

In [7]:
from sklearn.utils import resample
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [8]:
import torch
from datasets import Dataset, load_metric
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)

In [9]:
import torch
from datasets import Dataset, load_metric
import numpy as np


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = load_metric("f1")
    precision = load_metric("precision")
    recall = load_metric("recall")
    accuracy = load_metric("accuracy")

    f1_score = f1.compute(predictions=predictions, references=labels, average='binary')
    precision_score = precision.compute(predictions=predictions, references=labels, average='binary')
    recall_score = recall.compute(predictions=predictions, references=labels, average='binary')
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_score['accuracy'],
        "f1": f1_score['f1'],
        "precision": precision_score['precision'],
        "recall": recall_score['recall']
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  3%|▎         | 500/16000 [17:07<6:36:40,  1.54s/it] 

{'loss': 0.515, 'grad_norm': 9.450413703918457, 'learning_rate': 5e-05, 'epoch': 0.12}


  6%|▋         | 1000/16000 [29:54<6:56:11,  1.66s/it]

{'loss': 0.4468, 'grad_norm': 3.3079748153686523, 'learning_rate': 4.8387096774193554e-05, 'epoch': 0.25}


  9%|▉         | 1500/16000 [44:30<7:02:58,  1.75s/it]

...

{'loss': 0.2071, 'grad_norm': 0.17462489008903503, 'learning_rate': 0.0, 'epoch': 4.0}


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_

{'eval_loss': 0.4724868834018707, 'eval_accuracy': 0.891125, 'eval_f1': 0.8905091137649277, 'eval_precision': 0.8886101354741596, 'eval_recall': 0.892416225749559, 'eval_runtime': 336.0427, 'eval_samples_per_second': 23.806, 'eval_steps_per_second': 1.488, 'epoch': 4.0}
{'train_runtime': 25062.8416, 'train_samples_per_second': 5.107, 'train_steps_per_second': 0.638, 'train_loss': 0.34635793495178224, 'epoch': 4.0}


100%|██████████| 500/500 [05:26<00:00,  1.51it/s]c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument

{'eval_loss': 0.4724868834018707,
 'eval_accuracy': 0.891125,
 'eval_f1': 0.8905091137649277,
 'eval_precision': 0.8886101354741596,
 'eval_recall': 0.892416225749559,
 'eval_runtime': 333.2138,
 'eval_samples_per_second': 24.009,
 'eval_steps_per_second': 1.501,
 'epoch': 4.0}

In [10]:
model_path = "C:/Users/DELL/Downloads/modelmovie"  
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('C:/Users/DELL/Downloads/modelmovie\\tokenizer_config.json',
 'C:/Users/DELL/Downloads/modelmovie\\special_tokens_map.json',
 'C:/Users/DELL/Downloads/modelmovie\\vocab.json',
 'C:/Users/DELL/Downloads/modelmovie\\merges.txt',
 'C:/Users/DELL/Downloads/modelmovie\\added_tokens.json')